<a href="https://colab.research.google.com/github/hd77alu/ml-formative3/blob/main/ml_formative3_g6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Formative 3 - Probability Distributions, Bayesian Probability, and Gradient Descent Implementation

## Part 1: Probability Distributions



1. Using a relevant dataset sourced online, you will compute the probability density values for each data point using the bivariate normal distribution formula. Implement this from scratch without using any statistical libraries in Python.

2. Visualize the PDF with Matplotlib using a contour plot and a 3D plot.





## Part 2: Bayesian Probability

You will use [IMDb Movie Reviews](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) DatasetLinks to an external site. for this part of the assignment. Load the dataset using pandas

1. Keyword Selection

- As a group, choose 2–4 keywords that you believe indicate positive sentiment and 2–4 keywords that indicate negative sentiment.

2. Choice of Conditional Probability

- Decide whether your group will calculate P(Positive | keyword) or P(Negative | keyword) —
do not compute both.

3. Computation and Presentation

- For each chosen keyword, create a small table or markdown block showing the following probabilities:

- Prior: P(Positive)

- Likelihood: P(keyword|Positive)

- Marginal:  P(keyword)

- Posterior:  P(Positive|keyword)

4. Implementation

- Implement Bayes’ Theorem in Python to compute the posterior probability for each keyword based on your selected dataset.

- You may use basic Python operations only (no external machine learning libraries).

## Part 3: Gradient Descent Manual Calculation

- Objective:
You will manually compute three updates of the gradient descent algorithm for the parameters and in a simple linear regression model.

- Instructions:

1. Given the linear equation where: y = mx + b

- Initial m = -1
- Initial b = 1
- Learning rate = 0.1
- Given points: (1,3) and (3,6)

2. Compute the predicted values (y) for each data point using the current values of (m) and (b)

3. Derive the gradient of the cost function J(m,b), using Mean Squared Error (MSE). In Essence, show the calculation steps to arrive at the derivative of the cost function

4. Iteratively Update (m) and (b) using gradient descent

- The number of times you will update m and b is equal to the number of members that are in each respective group
- Show all calculations clearly and include intermediate results after each step.
- Each member must do at least 1 iteration Describe the trend you observe in the values of and. Are they moving towards reducing the error?

5. Submission:
Submit a neatly written  document with all calculations.

## Part 4: Gradient Descent in Code

1. Convert the manual calculations into Python code using SciPy.
2. Write code to :
- Update the values of m and b
- Compute predictions (the value of Y) using the final values of
 m and b.
3. Ensure the code does not abstract the update process excessively, meaning each step should be clearly visible.
4. Visualize how m, b, and the Error change over iterations using Matplotlib in two separate plots.

### Things to Note

Any Generic responses will result in a heavy penalty.

### Submission

- A GitHub Repository
- Jupyter Notebook with all implementations. Insert relevant images in the notebook to make an enticing presentation.
- A NEAT PDF showing handwritten Manual Calculations of Part 3
- A PDF file Showing Contributions

(Any code presented must be modular and follow the DRY principle)